# 0 Business Undestanding

## Motivação
### Qual o contexto?
O CFO requisitou essa solução em uma reunião de resultados mensais aos gerentes
## Causa raiz do problema
### Por que fazer uma previsão de venda?
Investimento em reformas nas lojas
## Dono do problema
### Quem será o stakeholder?
CFO / alguem que conheça o problema
## Formato da solução
### Qual formato da solução ?
* Granularidade - previsão em R$ nas proximas 6 semanas
* Tipo do problema - previsão de venda / demanda
* Potenciais métodos - time series - regressao - redes neurais - deep learning
* Formato da entrega - previsão das vendas nas proximas 6 semana via celular

## Data fields

* Id - an Id that represents a (Store, Date) duple within the test set
* Store - a unique Id for each store
* Sales - the turnover for any given day (this is what you are predicting)
* Customers - the number of customers on a given day
* Open - an indicator for whether the store was open: 0 = closed, 1 = open
* StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* StoreType - differentiates between 4 different store models: a, b, c, d
* Assortment - describes an assortment level: a = basic, b = extra, c = extended
* CompetitionDistance - distance in meters to the nearest competitor store
* CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* Promo - indicates whether a store is running a promo on that day
* Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

# Analise Descritiva

# 0.0 Imports

In [ ]:
import math
import pandas as pd
import inflection
import numpy as np

import seaborn as sns

from IPython.core.display import HTML

## 0.1 Helper Functions

In [ ]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
    
jupyter_settings()

## 0.2 Load Data

In [ ]:
df_sales_row = pd.read_csv("./data/train.csv", low_memory=False )
df_store_row = pd.read_csv("./data/store.csv", low_memory=False )

#merge
df_row = pd.merge(df_sales_row, df_store_row, how="left", on="Store")

In [ ]:
df_row.sample()

# 1.0 Descricao dos dados


## 1.1 Rename columns

In [ ]:
df1 = df_row.copy()

In [ ]:
df1.columns

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

snackcase = lambda x: inflection.underscore(x)

cols_new = list(map(snackcase, cols_old))

#rename
df1.columns = cols_new

## 1.2 Data Dimmensions


In [ ]:
print(f'Numero de linhas: {df1.shape[0]}')
print(f'Numero de colunas: {df1.shape[1]}')

## 1.3 Data types

In [ ]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

## 1.4 Check NA

## Formas de trabalhar com N/A
1 Exclusão: 
    * vantagem: rapido
    * desvantagem: perde informação
2 Machine learning 
    * substituição baseado no comportamento da coluna (media/ mediana/ clusterização)
3 Pensando no negócio
    * Como gerou este N/A? 
    

In [ ]:
df1.isna().sum()

## 1.5 Fillout NA

In [ ]:
df1.sample()

In [ ]:
#competition_distance 
## Se esta coluna esta NA pode ser que a distancia entre o concorrente é muito grande, não sendo reconhecida dentro do sistema como um concorrente.
## Para substituir este dado usaremos o um valor maior que a maior distancia presente nos dados, o que significa que não tem um competidor proximo 
df1['competition_distance'] = df1['competition_distance'].apply(lambda x : 200000.0 if math.isnan(x) else x)

#competition_open_since_month
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

#competition_open_since_year
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

#promo2_since_week               
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

#promo2_since_year
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

#promo_interval
month_map = {1:'Jan', 2:'Fev', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}

df1['promo_interval'].fillna(0, inplace=True)

df1['month_map'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [ ]:
df1.sample(5).T

In [ ]:
df1.isna().sum()

## 1.6 Data types

In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)
df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

In [ ]:
df1.dtypes

## 1.7 Descriptive Statistical

In [ ]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

In [ ]:
cat_attributes.sample(2)

### 1.7.1 Numerical Attributes

In [ ]:
#Central Tendency - mean, median
ct_mean = pd.DataFrame(num_attributes.apply(np.mean)).T
ct_median = pd.DataFrame(num_attributes.apply(np.median)).T

#Dispersion desvio padrão std, min, max, range, skew, kurtosis
dispersion_std = pd.DataFrame(num_attributes.apply(np.std)).T
dispersion_min = pd.DataFrame(num_attributes.apply(min)).T
dispersion_max = pd.DataFrame(num_attributes.apply(max)).T
dispersion_range = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
dispersion_skew = pd.DataFrame(num_attributes.apply(lambda x: x.skew() - x.min())).T
dispersion_kurt = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() - x.min())).T

#Concatenate
metrics = pd.concat(
    [dispersion_min, dispersion_max, dispersion_range, ct_mean, ct_median, dispersion_std, dispersion_skew, dispersion_kurt]
).T.reset_index()
metrics.columns = ['features','min','max','range','mean','median','standard','skew','kurtosis']

In [ ]:
metrics

In [ ]:
sns.distplot(df1['sales'])

## 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0])

In [ ]:
aux1 = df1[(df1['state_holiday'] != '0') & (df1['sales'] > 0)]

plt.subplot(1,3,1)
sns.boxplot(x='state_holiday' , y='sales' , data=aux1)

plt.subplot(1,3,2)
sns.boxplot(x='store_type' , y='sales' , data=aux1)

plt.subplot(1,3,3)
sns.boxplot(x='assortment' , y='sales' , data=aux1)